<center>

**Vježba 2 - Čišćenje IRIS skupa podataka**

</center>

U ovoj vježbi želimo vidjeti postupke čišćenja o kojima smo govorili na predavanjima. Nakon čišćenja upotrijebit ćeme neke algoritme strojnog učenja nad pročišćenim skupom.

Prvo skinemo IRIS skup podataka u CSV obliku.

In [ ]:
%%bash
wget -O iris.csv https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data

Load + brisanje kolona koje imaju manje od 1% jedinstvenih vrijednosti

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv('iris.csv', header = None)
X = df.iloc[:,:-1]
y = df.iloc[:,-1]

del df # Pazimo na memoriju... brišemo sve što nam više ne treba

dfn = X.nunique()
stats = pd.DataFrame()
stats['total'] = dfn
stats['perc'] = (stats['total']/X.shape[0])*100
plt.bar(x = X.columns, height = stats['perc'])
plt.title('Precentage of unique values')
plt.xlabel('Column')
plt.ylabel('Percentage')
plt.show()
plt.close()

print("Sparse ====")
sparse = stats[stats['perc'] < 1.0]
for i,row in sparse.iterrows():
    tot = row['total']
    print(f'Column {i} has {tot} unique values')

sparse_cols = list(sparse.index)
X = X.drop(columns = sparse_cols)
stats = stats.drop(index = sparse_cols)

print(f'New number of columns is {len(stats)}')

Obrišemo kolone s malom varijancom.

In [ ]:
from sklearn.feature_selection import VarianceThreshold
from numpy import arange

print(f'X shape {X.shape}')
print(f'y shape {y.shape}')

ths = arange(0.0, 0.55, 0.05)
res = []
for th in ths:
    trans = VarianceThreshold(threshold = th)
    X_selected = trans.fit_transform(X)
    res.append(X_selected.shape[1])

plt.plot(ths, res)
plt.show()

In [ ]:
trans = VarianceThreshold(threshold = 0.15)
X = trans.fit_transform(X) # Vratiti će se numpy array
X = pd.DataFrame(X)

Provjera duplih redaka.

In [ ]:
dups = X.duplicated()
duprows = dups.any()
print(f'Ima duplih redaka:{duprows}')
print(f'Prije brisanja {X.shape} {y.shape}')
if duprows:
    print(X[dups])
    X = X[~dups]
    y = y[~dups]

print(f'Poslije brisanja {X.shape} {y.shape}')

Korelacija značajki. Visoko korelirani se trebaju pregledati.

In [ ]:
import seaborn as sns
import numpy as np

X_corr = X.corr()
mask = np.triu(np.ones_like(X_corr, dtype = bool))
cmap = sns.diverging_palette(220, 10, as_cmap=True)

#make the heatmap plot
plt.figure(figsize=(2,2))
sns.heatmap(X_corr, mask=mask, cmap=cmap, vmax=.3, center=0, square=True, linewidths=.5, cbar_kws={"shrink": .5})
plt.tight_layout()
plt.show()
plt.close()

Skaliranje za ANN, te razdvajanje na train i test.

In [ ]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split

#scaler = MinMaxScaler()
scaler = StandardScaler()

X_scal = scaler.fit_transform(X)
X_scal = pd.DataFrame(X_scal)

X_train, X_test, y_train, y_test = train_test_split(X_scal, y, test_size = 0.2)
print(f'Train shapes {X_train.shape} {y_train.shape}')
print(f'Test shapes {X_test.shape} {y_test.shape}')

Provjera s MLP klasifikatorom.

In [ ]:
from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier(hidden_layer_sizes=(10,), activation='relu', solver='adam', max_iter=2000, verbose=1)
mlp.fit(X_train, y_train)

predict_train = mlp.predict(X_train)
predict_test = mlp.predict(X_test)

Ispis metrike poskupa za učenje i testiranje.

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

print(confusion_matrix(y_train,predict_train))
print(classification_report(y_train,predict_train))

print(confusion_matrix(y_test,predict_test))
print(classification_report(y_test,predict_test))

k-struko unakrsno testiranje skupa s dummy klasifikatorom (daje random izlaz), linearnim regresorom i MLP klasifikatorom. Ispisujemo točnost rezultata.

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn import metrics
import matplotlib.pyplot as plt

kfr = RepeatedStratifiedKFold(n_splits = 3, n_repeats = 2, random_state = 1)

reg1 = DummyClassifier()

vc = y.value_counts()
cw = {}
for i in vc.index:
    cw[i] = vc[i]/len(y)
print(f'Class weights {cw}')
#reg2 = LogisticRegression(solver = 'lbfgs', class_weight = cw)
reg2 = LogisticRegression(solver = 'liblinear')

mlp = MLPClassifier(hidden_layer_sizes=(10,), activation='relu', solver='adam', max_iter=2000)

score1 = cross_val_score(reg1, X_scal, y, scoring = 'accuracy', cv = kfr)
print(f'dummy {np.mean(score1)}')
score2 = cross_val_score(reg2, X_scal, y, scoring = 'accuracy', cv = kfr)
print(f'logistic regression {np.mean(score2)}')
score3 = cross_val_score(mlp, X_scal, y, scoring = 'accuracy', cv = kfr)
print(f'mlp {np.mean(score3)}')

Crtamo matricu konfuzije za prethodno k-struko unskrsno testiranje, za MLP klasifikator.
Crtamo za svaki prolaz.

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay


for fold,(train,test) in enumerate(kfr.split(X_scal,y)):
    mlp = MLPClassifier(hidden_layer_sizes=(10,), activation='relu', solver='adam', max_iter=2000)
    mlp.fit(X_scal.iloc[train], y.iloc[train])
    cm_mlp = metrics.confusion_matrix(y.iloc[test], mlp.predict(X_scal.iloc[test]))
    disp = ConfusionMatrixDisplay(confusion_matrix=cm_mlp, display_labels=y.unique())
    disp.plot(cmap=plt.cm.Wistia)
    plt.show()

Pokušavamo klasificirati s algoritmima za grupiranje. Ispisujemo ARI kao metriku.

In [ ]:
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
ycat = le.fit_transform(y)

kmclus = KMeans(n_clusters = 3, n_init = 'auto')
gm = GaussianMixture(n_components = 3)

score_km = cross_val_score(kmclus, X_scal, ycat, scoring = 'adjusted_rand_score', cv = kfr)
print(f'k-means ARI {np.mean(score_km)}')
score_gm = cross_val_score(gm, X_scal, ycat, scoring = 'adjusted_rand_score', cv = kfr)
print(f'GaussianMixture ARI {np.mean(score_gm)}')